# Ad-Hoc

## 1. Run Ad-Hoc

### 1.1. List servers in group

In [ ]:
ansible group_debian --list

### 1.2. Work with module

```bash
$ ansible <group1, group2, ..., groupn | all> -m <module> -a "command string"
```

For example:
```bash
$ ansible all -m shell -a "ps aux"
```

## 2. Modules

### 2.1. Ping

In [ ]:
ansible group_debian1 -m ping

### 1.2. Raw 

In [ ]:
ansible group_debian1 -m raw -a "ps aux"

### 1.3. Software repository

#### 1.3.1. Yum

#### 1.3.2. Apt 

### 1.4. Template

- Arguments
    - `src`: template file in current machine
    - `dest`: target file in remote server
    - `owner`: owner of target file
    - `mode`: mode of target file, canbe numbers (like `0644`) or letters (like `u=rw,g=r,o=r`), see also `chmod` command
    - `backup`: `ture` if create backup file if file content is changed

- Run template module

In [ ]:
ansible group_debian1 -m template \
    -a "src='./test_template.txt' dest='~/test_template.txt' owner='alvin' mode='u=rw,g=r,o=r' backup='yes'" \
    --extra-vars "dynamic_name=Alvin"

- Show generated files on remote server

In [ ]:
ansible group_debian1 -m raw -a "ls -al ~/test_template*"

- Show content of lastest generated file

In [ ]:
ansible group_debian1 -a "cat ~/test_template.txt"

- remove all generated files on remote server

In [ ]:
ansible group_debian1 -m raw -a "rm ~/test_template*"

### 1.5. Copy

- `src`: The source file on local machine, can be file or directory
- `content`: The content will be copied
- `dest`: The remote server to copied file
- `directory_mode`: The permission mode of remote directory
- `force`: Always copy file to target even file exist when value is 'yes'
- other arguments: all arguments for `file` module can use for this module, see `file` module

#### 1.5.1. Copy file

- Use copy module

In [ ]:
ansible group_debian1 -m copy -a "src='./ansible.cfg' dest='~/' force='yes'"

- Show copied file on remote server

In [ ]:
ansible group_debian1 -m raw -a "ls -al ~/ansible.*"

- Remove copied files

In [ ]:
ansible group_debian1 -m raw -a "rm ~/ansible.*"

#### 1.5.2. Copy files in folder

- Use copy module

In [ ]:
ansible group_debian1 -m copy -a "src='./hosts/' dest='~/target' force='yes'"

- Show copied folders

In [ ]:
ansible group_debian1 -m raw -a "ls -al ~/target"

- Remove copied files

In [ ]:
ansible group_debian1 -m raw -a "rm -rf ~/target*"

#### 1.5.3. Copy folder

- Use copy module

In [ ]:
ansible group_debian1 -m copy -a "src='./hosts' dest='~/target' force='yes'"

- Show copied folders

In [ ]:
ansible group_debian1 -m raw -a "ls -al ~/target"
ansible group_debian1 -m raw -a "ls -al ~/target/hosts"

- Remove copied files

In [ ]:
ansible group_debian1 -m raw -a "rm -rf ~/target*"

#### 1.5.4. Copy content

- Use copy module

In [ ]:
ansible group_debian1 -m copy -a "content='Hello World' dest='~/target.txt' force='yes'"

- Show copied folders

In [ ]:
ansible group_debian1 -m raw -a "cat ~/target.txt"

- Remove copied files

In [ ]:
ansible group_debian1 -m raw -a "rm target.txt"

### 1.6. User and Group

#### 1.6.1. User

- `home`: Home folder of user
- `groups`: Groups of user
- `uid`: Id of user
- `password`: Password of user, password cannot be plaintext, it need hashed by sha512
- `name`: User name
- `createhome`：`yes` if create home directory
- `system`: `yes` if a system user
- `remove`: `yes` if delete user home directory when `state = absent`
- `state`: Create (`statue=present`) or remove (`state=absent`) user
- `shell`: Which shell for this user
- `generate_ssh_key`: `yes` if create ssh key for this user
- `ssh_key_bits`: Length (bits) of ssh key
- `ssh_key_passphrase`: Phrase password of ssh key
- `ssh_key_file`: File name to save ssh key
- `ssh_key_type`: Type of ssh key

- Add and modify user: if user exist, then modify this user by new arguments

In [ ]:
ansible group_debian1_vault -m user \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "name=test \
        password={{ 'test'|password_hash('sha512') }} \
        append=yes \
        createhome=yes \
        shell=/bin/bash \
        generate_ssh_key=yes \
        groups=sudo,root \
        state=present"

- Show user and password (hashed) on remote server

In [ ]:
ansible group_debian1_vault -m raw \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "cat /etc/shadow" | grep 'test'

- Allow sudo for new user

In [ ]:
ansible group_debian1_vault -m copy \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "content='alvin ALL=(ALL:ALL) ALL\n' \
        dest='/etc/sudoers.d/alvin' \
        force='yes' \
        mode='u=r,g=r'"

- Copy ssh public key from current machine to remove server

In [ ]:
ansible group_debian1_vault -m authorized_key \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "user=test \
        key={{ lookup('file', '~/.ssh/id_rsa.pub') }} \
        state=present"

- Delete user

In [ ]:
ansible group_debian1_vault -m user \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "user=test \
        remove=yes \
        state=absent"

#### 1.6.2. Group

- `gid`：Id of group
- `name`: Name of group
- `state`: State of group, `present` or `absent`
- `system`: `yes` if a system group

- Create a group

In [ ]:
ansible group_debian1_vault -m group \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "name=test \
        state=present"

- Remove a group

In [ ]:
ansible group_debian1_vault -m group \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "name=test \
        state=absent"

### 1.7. Service

- `arguments`: Arguments for commandline
- `enabled`: `yes` if start server when user login
- `name`: Name of service
- `runlevel`: Level of service
- `sleep`: If `state` is `restarted`, the delay seconds between `stop` and `start`
- `state`: State of service, value can be: `started`, `stopped`, `restarted`, `reloaded`

- Stop service

In [ ]:
ansible group_debian1_vault -m service \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "name=sshd state=stopped"

- Start service

In [ ]:
ansible group_debian1_vault -m service \
    -b --vault-id=vault-id --extra-vars "@password.yml"  \
    -a "name=sshd state=started"